### Nesse arquivo obtenho um polinomio mais simples. Rodo a ideia da particao colocando as raizes no maximos e minimos.

##### Nesse arquivo rodo bem a ideia da particao. No entanto uma raiz para t=1 impede a finalizacao. Tentando solucionar.
##### Aparentemente nao da certo inverter o intervalo para deixar semiaberto no 1.

In [22]:
load("../util.sage")
C1 = load('blocks/linear_block_1')
C1 = C1.apply_map(lambda x: x.simplify_real().combine()); show(C1)
p1 = simplify_expression(C1.det())

[                                                                                                                                     24*c - 1/12*sqrt(3) - 3/8*sqrt(2) - 3/4       24*c - 1/3*(sqrt(3)*t + sqrt(3))/(t^3 + 3*t^2 + 3*t + 1) - 3*(t + 3)/(3*t^2 + 2*t + 3)^(3/2) + 3*(t - 3)/(3*t^2 - 2*t + 3)^(3/2) - 1/3*(sqrt(3)*t - sqrt(3))/(t - 1)^3]
[24*c*t - 1/3*(sqrt(3)*t + sqrt(3))/(t^3 + 3*t^2 + 3*t + 1) - 3*(3*t + 1)/(3*t^2 + 2*t + 3)^(3/2) - 3*(3*t - 1)/(3*t^2 - 2*t + 3)^(3/2) + 1/3*(sqrt(3)*t - sqrt(3))/(t - 1)^3                                                                                                                               24*c*t - 1/24*(2*sqrt(3) + 9*sqrt(2) + 18)/t^2]

In [23]:
c, t =var('c t');
assume(t>0)
assume(t<1)
#t = var('t')
def extremos_uv_particionados(p , extremos_t):
    """ INPUT - UM POLINOMIO QUADRATICO NA VARIAVEL t DA FORMA
   p --> p(t)=a*t^2 + b*t+c 
   extremos_t ----> Os extremos para o t, (deve ser um subintervalo do INTERVALO [0,1])
   OUTPUT - OS VALORES MAXIMOS E MINIMOS DE p NO INTERVALO [t0,t1].
    """
    t0 =  extremos_t[0]
    t1 =  extremos_t[1]
    coefs = reversed(p.coefficients(t))
    a, b , c = map(lambda x: x[0], coefs )
    
    if t0 <=-b/(2*a) <=t1:
        
        lst = [p(t=t0), p(t=t1), p(t=-b/(2*a))]
        
    else:
        lst = [p(t=t0), p(t=t1)]
        
    return min(lst), max(lst)

###TESTAR SE f e g é um par u e v.
def teste_par_u_v(lista):
    """INPUT- Uma lista com o possivel par f e g para v e u(nesta ordem).
    OUTPUT- Um booleano Falso ou verdadeiro se f e g forma um par para a substituição"""
    f_lista = lista[0]
    
    g_lista = lista[1]
    
    b = (g_lista.coefficients()[1][0])
    
    return bool(((g_lista-f_lista)/(2*b)).full_simplify()== t)



In [24]:

poly_u_v_ico_num =load('poly_C11')

In [25]:
### Respeitar a ordem da variaveis eh importante
print(poly_u_v_ico_num.free_variables())
print(poly_u_v_ico_num.variables())

(t, u, v)
(t, u, v)


In [26]:
graus_u_v_t  =  [poly_u_v_ico_num.degree(t),poly_u_v_ico_num.degree(u), poly_u_v_ico_num.degree(v)]
print(graus_u_v_t)

[24, 4, 4]


In [27]:
from functools import reduce

def dicionario_geral(poly):
    """INPUT - poly- Um polinomio como expressão simbolica na classe <class 'sage.symbolic.expression.Expression'>
    OUTPUT - Um dicionario com CHAVE do tipo tupla multigrau (d1,d2,...,dl) correpondendo ao monomio 
    variavel1^(d1)*variavel2^(d2)*...*variavell^(dl) e VALOR sendo o coeficiente deste monomio.
    OBSERVACAO- A ordem das variaveis imspota pelo SAGE manda na ordem das chaves do dicionario. 
    Exemplo-
    sage: z, y, x, u = var('z y x u')
    sage: p = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = p.free_variables(); variaveis
        (u,x,y,z)
    sage: dicionario_geral(p)
        {(0, 2, 0, 2): 202,
         (1, 1, 0, 1): 1101,
         (2, 0, 2, 0): 2020,
         (2, 1, 0, 0): 2100,
         (3, 0, 2, 2): 3022,
         (3, 0, 3, 1): 3031}
        
        
    sage: u,v = var('u v')
    sage: p3 = 23*u^2*v^3+21*u^2*v+11*u*v+10*u+4;
    sage: dicionario_geral(p3)
      {(0, 0): 4, (1, 0): 10, (1, 1): 11, (2, 1): 21, (2, 3): 23}
    """
    def nova_lista(lista, var):
        new_lista = [];
        for item in lista:
            sublista = item[0].coefficients(var)
            for subitem in sublista:
                subitem = [subitem[0],subitem[1:]+item[-1]]
                new_lista.append(subitem)
        return  new_lista

    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
#####################################################################
    variavel_1 = variaveis[0];
    l_variavel_1 = (poly).coefficients(variavel_1);
    l = [[item[0],item[1:]] for item in l_variavel_1]
    for var in variaveis[1:]:
        l = nova_lista(l, var)
    dicionario = {tuple(list(reversed(j[1]))) : j[0] for j in l}
    return dicionario

#novo_coefsxy = dicionario_geral(poly_u_v) # virar variável global
def coeficientes_de_moebius_simbolico_varias_variaveis(poly, graus, extremos, novo_coefsxy ):
    """INPUTS -
    poly- Um polinomio como expressão simbolica na classe  <class 'sage.symbolic.expression.Expression'>;
    graus - Uma lista com naturais representando o multigrau do monomio que voce quer o coeficiente.
            Importante - A sua lista de graus deve estar na ordem que o Sage ordena as variaveis.
    extremos - Uma lista com pares (a_i,b_i) que dão o intervalo da substituição de Moebius para
    a variavel x_i, isto é,    x_i---> x_i = (a_i*u_i+b_i)/(u_i+1) para a variavel u_i.
    OUTPUT - O coeficiente do monomio com multigrau dado pela lista graus depois da substituicao de Moebius
    Exemplo - 1
    sage: poly_teste = 23*u^2*v^3 + 21*u^2*v + 11*u*v + 10*u + 4;
    sage: graus_teste = [1,2]; ## para acahar o coeficiente de (x_1)^1*(x_2)^2
    sage: extremos_teste = [(3, 5), (7, 11)];
    sage: novo_coefsxy_teste = dicionario_geral(poly_teste); # virar variável global;
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly_teste, graus_teste, extremos_teste, novo_coefsxy_teste)
        1133944
    Exemplo - 2
    sage: poly = (2100*u^2*x+2020*u^2*y^2)*z^0+(3031*u^3*y^3+1101*u*x)*z+(3022*u^3*y^2+202*x^2)*z^2
    sage: variaveis = poly.free_variables(); variaveis
    sage: extremos_teste_2 = [(3,5),(7,11),(13,17),(19,23)];
    sage: ru, rx, ry, rz = var('ru, rx, ry, rz')
    sage: variaveis_r = [ru, rx, ry, rz]
    sage: novo_coefsxy_teste_2 = dicionario_geral(poly); 
    sage: c = coeficientes_de_moebius_simbolico_varias_variaveis(poly, [0, 0, 1, 0], extremos_teste_2, novo_coefsxy_teste_2) ## Procurando o coeficiente de ru^0 * rx^0 * ry^1 * rz^0
    sage: show(c)
     244374065874
       
       OBSERVACAO: Para mais detalhes veja o arquivo /Artigo_de_Aplicacoes/Reuniao-casa-marcelo-26-07/Icosaedro-bloco-2-reunião-casa-marcelo/Funcao_Para_Coeficientes_Com_Varias_Variaveis_Versao_Final.ipynb
        """
    #variaveis = (poly).free_variables(); Modifiquei aqui
    variaveis = (poly).variables()
    graus_total = [poly.degree(x) for x in variaveis];
    chaves = novo_coefsxy.keys()
    soma_total = 0
    for chave in chaves:
        somas = []
        for k in range(len(chave)):
            somak = 0
            ck = chave[k]
            gt = graus_total[k]
            gks = graus[k]
            a_k = extremos[k][0]
            b_k = extremos[k][1]
            for pp in range(ck+1):
                somak = somak+((binomial(ck,pp))*(binomial(gt-ck, gks-pp))*((a_k)^(pp))*((b_k)^(ck-pp)))
            somas.append(somak)
        somas_e_valor = somas + [novo_coefsxy[chave]]
        soma_total = soma_total + reduce(lambda a, b: a*b, somas_e_valor)
    return soma_total
### Alias
cmsvv = coeficientes_de_moebius_simbolico_varias_variaveis


In [28]:
n_uplas = [[i,j,k] for i in range(graus_u_v_t[0]+1) for j in range(graus_u_v_t[1]+1) for k in range(graus_u_v_t[2]+1)]

In [29]:
len(n_uplas)

625

In [30]:
%time
dicionario = dicionario_geral(poly_u_v_ico_num);

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.62 µs


### Adicionando Ideia da Particao

In [31]:
### Vou fazer gambiarra para incluir a curva inicial 
def extremos_parcial_curvas(intervalo, curvas):
    """INPUT - Um intervalo, no formato de lista [a,b] ou tupla [a,b]. Com a<b.
    OUTPUT - Os  extremos de uma caixa que contem o traço da curva parametrizada
    por t---> (t, g(t), h(t)), para t variando no intervalo [a,b].
    """
    ### Caso geral o t_0 =-b/(2*a) do polinomio do segundo grau que representa as variaveis
    m = intervalo[0];
    M = intervalo[1];
    extremos_parcial = [];
    for curva in curvas:
        coe = curva.coefficients()
        c,b,a = coe[0][0],coe[1][0],coe[2][0];
        t_0 = -b/(2*a);
        if m <= t_0 and t_0 <= M:
            minimo = min([curva(t = m), curva(t = t_0), curva(t = M)])
            maximo = max([curva(t = m), curva(t = t_0), curva(t = M)])
        elif t_0 <= m and M <= t_0:
            minimo = min([curva(t = m), curva(t = t_0), curva(t = M)])
            maximo = max([curva(t = m), curva(t = t_0), curva(t = M)])
        else:
            minimo = min([curva(t = m), curva(t = M)])
            maximo = max([curva(t = m), curva(t = M)])
        extremos_parcial.append((sqrt(minimo),sqrt(maximo))) #Coloquei raiz
    return extremos_parcial


def particao5(lista_de_intervalos, curvas):
    """INPUT - Lista de Pares do tipo [((a_1,b_1), False), ...,((a_n,b_n), True)] 
    Onde o intervalo i será usado se o seu par for False, e não será usado se seu par for True.
    No caso False, calcula a mudança de coeficientes e se todos tiverem o mesmo sinal muda o False para True.
    Caso haja mudança de sinal nos coeficientes bissecta o intervalo e coloca False nos dois intervalos gerados.
    """
    um_e_menos_um = Set({1,-1});
    n_l_i = []
    for i in range(len(lista_de_intervalos)):
        if lista_de_intervalos[i][-1]:
            n_l_i.append(lista_de_intervalos[i])
            continue
        else:
            intervalo = lista_de_intervalos[i][0];
            m = intervalo[0];
            M = intervalo[1];
            e_p = extremos_parcial_curvas(intervalo, curvas); ###Mudou aqui [f,g,h] trocou por curvas.
            e_p = [(m,M)] + e_p ### Gambiarra aqui para nao incluir a curva t no codigo
            coefs = Set([]);
            n_l_i = n_l_i +[((m,M),True)]
            for graus in n_uplas:
                ultimo_coeficiente = cmsvv(poly, graus, e_p, dicionario);
                u_c = Set([sign(ultimo_coeficiente)]);
                coefs = coefs.union(u_c);
                if um_e_menos_um.issubset(coefs):
                    n_l_i = n_l_i[:-1] + [((m,(1/2)*(m+M)), False), (((1/2)*(m+M),M), False)];
                    break
    return n_l_i

In [60]:
### Curvas
f(t) = t;
f_1t(t) = 3*t^2 + 2*t + 3 # v1**2
f_2t(t) = 3*t^2 - 2*t + 3 # v2**2


In [61]:
curvas = [f_1t, f_2t]
poly = poly_u_v_ico_num

In [62]:
#### RODA ATE AQUI E PAUSA

In [63]:
%time s5 = particao5([((0, 1), False)], curvas);

CPU times: user 35.5 s, sys: 39.9 ms, total: 35.5 s
Wall time: 35.5 s


In [64]:
print(s5)

[((0, 1/2), False), ((1/2, 1), False)]


In [65]:
%time s6 = particao5(s5, curvas)

CPU times: user 1min 23s, sys: 89.9 ms, total: 1min 23s
Wall time: 1min 23s


In [66]:
print(s6)

[((0, 1/4), False), ((1/4, 1/2), False), ((1/2, 1), True)]


In [67]:
%time s7 = particao5(s6, curvas)

CPU times: user 1min 24s, sys: 39.8 ms, total: 1min 24s
Wall time: 1min 24s


In [68]:
print(s7)

[((0, 1/8), False), ((1/8, 1/4), False), ((1/4, 1/2), True), ((1/2, 1), True)]


In [71]:
%time s8 = particao5(s7, curvas)

CPU times: user 1min 36s, sys: 49.9 ms, total: 1min 36s
Wall time: 1min 36s


In [72]:
print(s8)

[((0, 1/16), False), ((1/16, 1/8), False), ((1/8, 1/4), True), ((1/4, 1/2), True), ((1/2, 1), True)]


In [73]:
%time s9 = particao5(s8, curvas)

CPU times: user 1min 23s, sys: 29.9 ms, total: 1min 23s
Wall time: 1min 23s


In [74]:
print(s9)

[((0, 1/32), False), ((1/32, 1/16), False), ((1/16, 1/8), True), ((1/8, 1/4), True), ((1/4, 1/2), True), ((1/2, 1), True)]


In [33]:
%time s10 = particao5(s9, curvas)

CPU times: user 1min 29s, sys: 69.9 ms, total: 1min 29s
Wall time: 1min 29s


In [34]:
print(s10)

[((0, 1/64), False), ((1/64, 1/32), False), ((1/32, 1/16), True), ((1/16, 1/8), True), ((1/8, 1/4), True), ((1/4, 1/2), True), ((1/2, 1), True)]


In [35]:
%time s11 = particao5(s10, curvas)

CPU times: user 1min 54s, sys: 29.9 ms, total: 1min 54s
Wall time: 1min 54s


In [36]:
print(s11)

[((0, 1/128), False), ((1/128, 1/64), False), ((1/64, 1/32), True), ((1/32, 1/16), True), ((1/16, 1/8), True), ((1/8, 1/4), True), ((1/4, 1/2), True), ((1/2, 1), True)]


In [38]:
%time s12 = particao5(s11, curvas)

CPU times: user 1min 54s, sys: 49.9 ms, total: 1min 54s
Wall time: 1min 54s


In [39]:
print(s12)

[((0, 1/256), False), ((1/256, 1/128), False), ((1/128, 1/64), True), ((1/64, 1/32), True), ((1/32, 1/16), True), ((1/16, 1/8), True), ((1/8, 1/4), True), ((1/4, 1/2), True), ((1/2, 1), True)]


In [44]:
%time s13 = particao5(s12, curvas)

CPU times: user 1min 47s, sys: 70 ms, total: 1min 47s
Wall time: 1min 47s


In [45]:
print(s13)

[((0, 1/512), False), ((1/512, 1/256), False), ((1/256, 1/128), True), ((1/128, 1/64), True), ((1/64, 1/32), True), ((1/32, 1/16), True), ((1/16, 1/8), True), ((1/8, 1/4), True), ((1/4, 1/2), True), ((1/2, 1), True)]


In [47]:
%time s14 = particao5(s13, curvas)

CPU times: user 1min 29s, sys: 70 ms, total: 1min 29s
Wall time: 1min 29s


In [48]:
print(s14)

[((0, 1/1024), False), ((1/1024, 1/512), False), ((1/512, 1/256), True), ((1/256, 1/128), True), ((1/128, 1/64), True), ((1/64, 1/32), True), ((1/32, 1/16), True), ((1/16, 1/8), True), ((1/8, 1/4), True), ((1/4, 1/2), True), ((1/2, 1), True)]


In [46]:
### Fato importante. Isso garante que o ultimo subintervalo continuara sempre subdividindo.
poly(t=0, u = sqrt(f_1t(0)), v = sqrt(f_2t(0)))

0

#### Tentando inverter o lado.

In [27]:
%time s5inv = particao5([((1,31/32), False)], curvas);

CPU times: user 3min 7s, sys: 196 ms, total: 3min 7s
Wall time: 3min 7s


In [28]:
show(s5inv)

[((1, 63/64), False), ((63/64, 31/32), False)]

In [29]:
%time s6inv = particao5(s5inv, curvas);

CPU times: user 10min 36s, sys: 836 ms, total: 10min 37s
Wall time: 10min 37s


In [30]:
show(s6inv)

[((1, 127/128), False), ((127/128, 63/64), False), ((63/64, 31/32), True)]

In [31]:
%time s7inv = particao5(s6inv, curvas);

CPU times: user 10min 34s, sys: 764 ms, total: 10min 34s
Wall time: 10min 34s


In [32]:
show(s7inv)

[((1, 255/256), False),
 ((255/256, 127/128), False),
 ((127/128, 63/64), True),
 ((63/64, 31/32), True)]

### Pra baixo eh lixo

In [37]:
show(polinomio_simples( v1 = sqrt(f_1t(1)), u1 = sqrt(g_1t(1)), v2 = sqrt(f_2t(1)), u2 = sqrt(g_2t(1))))

-10995116277760*sqrt(5)*t^4*(sqrt(5) - 3)^3*(sqrt(5) - 5)^3 + 32985348833280*t^4*(sqrt(5) - 3)^3*(sqrt(5) - 5)^3 + 21990232555520*sqrt(5)*t^3*(sqrt(5) - 3)^3*(sqrt(5) - 5)^3 - 65970697666560*t^3*(sqrt(5) - 3)^3*(sqrt(5) - 5)^3 - 10995116277760*sqrt(5)*t^2*(sqrt(5) - 3)^3*(sqrt(5) - 5)^3 + 32985348833280*t^2*(sqrt(5) - 3)^3*(sqrt(5) - 5)^3

In [38]:
p=polinomio_simples( v1 = sqrt(f_1t(1)), u1 = sqrt(g_1t(1)), v2 = sqrt(f_2t(1)), u2 = sqrt(g_2t(1)));

In [41]:
show(p.full_simplify().factor())

-3518437208883200*(t - 1)^2*t^2*(199*sqrt(5) - 445)